# Slice experiments - Spread Analysis

## Introduction

This notebook contains code to analyze fluorescence signals recorded during slice experiments, specifically used for two different types of experiments: **iGluSnFR** and **dLight** imaging. These experiments are designed to measure glutamate and dopamine release, respectively, using genetically encoded sensors. The fluorescence signal recorded from these sensors is processed to calculate $\Delta F/F$ (delta F over F; dff), and activation areas are determined based on the fluorescence response to various stimuli or drug treatments.

### Data Structure Overview

The data in these experiments is organized into two distinct structures based on the type of experiment:
1. **Quinpirole Drug Treatment Experiments:**

In this set of experiments, the fluorescence signal was recorded during different phases of a drug treatment protocol involving Quinpirole, a dopamine receptor agonist. The data is structured in folders representing different phases of the experiment:

- *Baseline*: The initial condition before drug treatment.
- *Treatment*: The phase during which the drug (Quinpirole) was applied.
- *Wash*: The recovery phase after the drug was washed out.

Each of these folders contains multiple TIFF stacks representing the fluorescence signal from various sweeps. These TIFF files are labeled as:

- `hiX.tif`: High-intensity fluorescence recordings.
- `loX.tif`: Low-intensity fluorescence recordings (optional, not present in some experiments. Refer to the methods of the paper).
- `noX.tif`: Control or baseline fluorescence.

$X$ represents the sweep number. The dF/F is calculated by comparing the high and low signals to the baseline, and the activation area is determined based on a threshold derived from the fluorescence variability. The threshold is set to $ 4 \times STD(Frames[:30])$


2. **Stimulus-Response (Stim-Response) Experiments:**

In these experiments, the slices were electrically stimulated, and the fluorescence response to the stimulation was measured. The data is organized in folders representing different stimulus intensities, labeled by their current (in microamperes, uA). Each folder contains fluorescence recordings for a specific stimulation intensity:

different current intensities used in the experiment:    `50uA`, `60uA`, `70uA`, `80uA`, `100uA`, `200uA`, `300uA`, `400uA`, `500uA`

Each uA folder contains multiple TIFF stacks labeled similarly to the drug treatment experiments:

- `hiX.tif`: High-intensity fluorescence recordings.
- `loX.tif`: Low-intensity fluorescence recordings.
- `noX.tif`: Control or baseline fluorescence.

$X$ represents the sweep number. The dF/F is calculated by averaging the responses across all the sweeps in each folder, and the activation area is calculated based on the same thresholding method.
Purpose of the Analysis

The primary goal of this analysis is to calculate and visualize the dF/F signal across the experimental conditions and determine the activation areas based on the fluorescence response. The code processes both Quinpirole drug treatment experiments and stim-response experiments, calculates the dF/F for each set of TIFF stacks, and generates 3D plots to visualize the average fluorescence response. Additionally, the activation areas for each set of conditions are saved in an Excel sheet for further analysis.

## Analysis of Quinpirole Experiments

This section is for analyzing the activated area over time following stimulation (Activation Volume). The output is an excel spreadsheet, with each column representing an experiment, and each row a sweep. The cells are color coded:
- No Fill: for the baseline sweeps
- Green: for Quin drug treatment
- Grey: for final aCSF wash

In [9]:
import os
import numpy as np
import tifffile as tiff
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill


# Helper function for natural sorting of file names
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]


def process_directory(root_folder):
    # Create a master dictionary to store the final results
    master_data = {}

    # Go through each folder (root directories containing 'Baseline *', 'Treatment *', 'Wash *')
    for folder in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder)
        if not os.path.isdir(folder_path):
            continue

        sub_dirs = ['Baseline', 'Treatment', 'Wash']
        sweep_data = []

        # Process each subdirectory (Baseline, Treatment, Wash)
        for sub_dir in sub_dirs:
            # Find the subdirectory that starts with 'Baseline', 'Treatment', or 'Wash'
            matching_sub_dirs = [d for d in os.listdir(folder_path) if d.startswith(sub_dir)]
            if not matching_sub_dirs:
                continue

            # There should be only one matching subdirectory, use it
            sub_dir_path = os.path.join(folder_path, matching_sub_dirs[0])
            print(sub_dir_path)
            # Gather all the hi, lo, and no tiff files
            hi_files = sorted([f for f in os.listdir(sub_dir_path) if f.startswith('hi') and f.endswith('.tif')], key=natural_sort_key)
            lo_files = sorted([f for f in os.listdir(sub_dir_path) if f.startswith('lo') and f.endswith('.tif')], key=natural_sort_key)
            no_files = sorted([f for f in os.listdir(sub_dir_path) if f.startswith('no') and f.endswith('.tif')], key=natural_sort_key)

            for i, hi_file in enumerate(hi_files):
                hi_stack = np.array(tiff.imread(os.path.join(sub_dir_path, hi_file)),dtype=np.float32)
                no_stack = np.array(tiff.imread(os.path.join(sub_dir_path, no_files[i])),dtype=np.float32)

                if i < len(lo_files):
                    lo_stack = np.array(tiff.imread(os.path.join(sub_dir_path, lo_files[i])),dtype=np.float32)
                    dff_stack = (hi_stack - no_stack) / no_stack + (lo_stack - no_stack) / no_stack
                    dff_stack /= 2  # Averaging the two dff calculations
                else:
                    dff_stack = (hi_stack - no_stack) / no_stack

                total_activation_area = calculate_activation_area(dff_stack)
                sweep_data.append({
                    'sweep': f'Sweep_{i+1}',
                    'sub_dir': sub_dir,
                    'activation_area': total_activation_area
                })

        # Add data to master dictionary with folder name as key
        master_data[folder] = sweep_data

    # Save the data into an Excel file with folder-based columns and color formatting
    save_excel(master_data, os.path.join(root_folder, 'master_dff_results_colored.xlsx'))


def calculate_activation_area(dff_stack):
    # Calculate the threshold for each pixel based on the first 30 frames
    std_dev = np.std(dff_stack[:30], axis=0)
    threshold = 4 * std_dev

    # Count how many pixels are above the threshold for each frame
    activated_area = np.sum(dff_stack > threshold, axis=(1, 2))

    # Total area activated across all frames
    total_activation_area = np.sum(activated_area)
    return total_activation_area


def save_excel(master_data, output_path):
    wb = Workbook()
    ws = wb.active
    ws.title = "dFF Results"

    # Define the colors for each subdirectory
    baseline_fill = PatternFill(start_color="FFFFFF", end_color="FFFFFF", fill_type="solid")  # No Fill
    treatment_fill = PatternFill(start_color="00FF00", end_color="00FF00", fill_type="solid")  # Green
    wash_fill = PatternFill(start_color="808080", end_color="808080", fill_type="solid")  # Grey

    # Write header
    ws.append(['Sweep'] + list(master_data.keys()))

    # Find maximum number of sweeps across all folders
    max_sweeps = max([len(data) for data in master_data.values()])

    # Fill the Excel file with data
    for row_index in range(max_sweeps):
        row_data = [f'Sweep_{row_index+1}']
        for folder, sweeps in master_data.items():
            if row_index < len(sweeps):
                sweep_info = sweeps[row_index]
                activation_area = sweep_info['activation_area']
                sub_dir = sweep_info['sub_dir']

                # Append activation area
                row_data.append(activation_area)

                # Apply the color based on the subdirectory
                if sub_dir.startswith('Baseline'):
                    fill = baseline_fill
                elif sub_dir.startswith('Treatment'):
                    fill = treatment_fill
                else:
                    fill = wash_fill

                # Apply fill color to the corresponding cell
                ws.cell(row=row_index + 2, column=len(row_data)).fill = fill
            else:
                # If there are no more sweeps for this folder, leave the cell empty
                row_data.append('')

        ws.append(row_data)

    # Save the workbook
    wb.save(output_path)
    print(output_path)



In [ ]:
# Analyze directory
root_folder = '/mnt/team/Raymond Lab/Judy/iGluSnFR Experiments/0.5uM quinpirole/WT'
process_directory(root_folder)

### Plotting the Area over time (Activation Volume)

This section plots the activation volume for the average of last 6 sweeps of each condition (Baseline, Treatment, Wash) for each experiment, and saves them as 300dpi png.

In [4]:
import os
import re
import numpy as np
import tifffile as tiff
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.colors import Normalize

# Helper function for natural sorting of file names
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

# Plot function
def plot_dff_stack_with_correct_transparency(sweep_stack, threshold, title, save_path):
    frames, height, width = sweep_stack.shape
    x = np.arange(width)
    y = np.arange(height)
    X, Y = np.meshgrid(x, y)

    # Create the 3D plot
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Normalize the colormap to the desired range
    norm = Normalize(vmin=0, vmax=0.2)

    # Define z scaling to improve the spread of the plot in the Z-axis (time)
    z_scale = 1  # Adjust this if the Z-axis looks squished

    frame_step = 5  # Skipping frames to speed up (adjust based on your data size)
    
    for t in range(0, frames, frame_step):
        frame = sweep_stack[t]

        # Apply transparency: Create a facecolors array based on dF/F values and mask
        facecolors = cm.jet(norm(frame))
        
        # Masking: Set alpha to 0 for values below the threshold (transparent), 0.6 for above, and 0.8 for way above, so that the core of the activated volume is clearly visible
        below_threshold_mask = frame < threshold
        above_threshold_mask = frame >= threshold
        way_above_threshold_mask = frame >= 3*threshold
        facecolors[below_threshold_mask, 3] = 0  # Set alpha channel to 0 where masked
        facecolors[above_threshold_mask, 3] = 0.6
        facecolors[way_above_threshold_mask, 3] = 0.8
        
        # Use plot_surface with custom facecolors
        ax.plot_surface(X, np.ones((height, width)) * t * z_scale, Y,
                        facecolors=facecolors, rstride=1, cstride=1, 
                        linewidth=0, antialiased=True)

    # Set axis labels and title
    ax.set_ylabel('Time (Frames)')
    ax.set_title(title)

    # Adjust the Z-axis limit to make sure it looks well-proportioned
    ax.set_ylim(0, frames * z_scale)

    # Add color bar with the new limits (0 to 0.1)
    mappable = cm.ScalarMappable(norm=norm, cmap='jet')
    fig.colorbar(mappable, ax=ax, shrink=0.5, aspect=5, label="dF/F Value")

    # Save the plot
    plt.savefig(save_path, dpi=300)
    
    # Show the plot (optional, can be removed if you only want to save)
    # plt.show()
    plt.close()


# Main function to process the data and plot the average of last 6 dF/F sweeps
def process_directory_with_dff_plot(root_folder):
    # Go through each folder (root directories containing 'Baseline *', 'Treatment *', 'Wash *')
    for folder in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder)
        if not os.path.isdir(folder_path):
            continue

        sub_dirs = ['Baseline', 'Treatment', 'Wash']
        
        # Process each subdirectory (Baseline, Treatment, Wash)
        for sub_dir in sub_dirs:
            matching_sub_dirs = [d for d in os.listdir(folder_path) if d.startswith(sub_dir)]
            if not matching_sub_dirs:
                continue

            sub_dir_path = os.path.join(folder_path, matching_sub_dirs[0])

            # Gather and sort hi, lo, and no tiff files using natural sorting
            hi_files = sorted([f for f in os.listdir(sub_dir_path) if f.startswith('hi') and f.endswith('.tif')], key=natural_sort_key)
            lo_files = sorted([f for f in os.listdir(sub_dir_path) if f.startswith('lo') and f.endswith('.tif')], key=natural_sort_key)
            no_files = sorted([f for f in os.listdir(sub_dir_path) if f.startswith('no') and f.endswith('.tif')], key=natural_sort_key)

            dff_values = []
            for i, hi_file in enumerate(hi_files):
                hi_stack = np.array(tiff.imread(os.path.join(sub_dir_path, hi_file)),dtype=np.float32)
                no_stack = np.array(tiff.imread(os.path.join(sub_dir_path, no_files[i])),dtype=np.float32)
                
                if i < len(lo_files):
                    lo_stack = np.array(tiff.imread(os.path.join(sub_dir_path, lo_files[i])),dtype=np.float32)
                    dff_stack = (hi_stack - no_stack) / no_stack + (lo_stack - no_stack) / no_stack
                    dff_stack /= 2  # Averaging the two dff calculations
                    
                else:
                    dff_stack = (hi_stack - no_stack) / no_stack
                    
                dff_values.append(dff_stack)

            # Calculate the average of the last 6 dF/F sweeps
            avg_dff = np.mean(dff_values[-6:], axis=0)

            # Calculate the threshold (4 times standard deviation of the first 30 frames)
            std_dev = np.std(avg_dff[:30], axis=0)
            threshold = 4 * std_dev

            # Define the path to save the plot
            plot_save_path = os.path.join(sub_dir_path, f"{sub_dir}_average_dff_plot.png")

            # Plot the average dF/F stack and save it
            plot_dff_stack_with_correct_transparency(avg_dff, threshold, f"{sub_dir} Average dF/F Plot", plot_save_path)



In [ ]:
# Call the process function with the root folder path
root_folder = '/mnt/team/Raymond Lab/Judy/dLight Experiments/WT/0.5uM quinpirole'
process_directory_with_dff_plot(root_folder)

## Analysis of Stim-Response Experiments

This section is for analysis of activated area following stimulations at different intensities.

In [15]:
import os
import re
import numpy as np
import tifffile as tiff
import pandas as pd

# Helper function for natural sorting of file names
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

# Function to calculate dF/F
def calculate_dff(hi_stack, lo_stack, no_stack):
    if lo_stack is not None:
        dff_stack = (hi_stack - no_stack) / no_stack + (lo_stack - no_stack) / no_stack
        dff_stack /= 2  # Averaging the two dff calculations
    else:
        dff_stack = (hi_stack - no_stack) / no_stack
    return dff_stack

# Function to calculate activation area based on dF/F thresholding
def calculate_activation_area(dff_stack):
    std_dev = np.std(dff_stack[:30], axis=0)
    threshold = 4 * std_dev # Threshold set to 4 times standard dev of first 30 frames (as before)
    activated_area = np.sum(dff_stack > threshold, axis=(1, 2))
    total_activation_area = np.sum(activated_area)
    return total_activation_area

# Function to process the uA folder and calculate the average dF/F and activation area
def process_uA_folder(uA_folder_path):
    # Gather and sort hi, lo, and no tiff files using natural sorting
    hi_files = sorted([f for f in os.listdir(uA_folder_path) if f.startswith('hi') and f.endswith('.tif')], key=natural_sort_key)
    lo_files = sorted([f for f in os.listdir(uA_folder_path) if f.startswith('lo') and f.endswith('.tif')], key=natural_sort_key)
    no_files = sorted([f for f in os.listdir(uA_folder_path) if f.startswith('no') and f.endswith('.tif')], key=natural_sort_key)

    dff_values = []
    for i, hi_file in enumerate(hi_files):
        hi_stack = np.array(tiff.imread(os.path.join(uA_folder_path, hi_file)),dtype=np.float32)
        no_stack = np.array(tiff.imread(os.path.join(uA_folder_path, no_files[i])),dtype=np.float32)

        lo_stack = np.array(tiff.imread(os.path.join(uA_folder_path, lo_files[i])),dtype=np.float32) if i < len(lo_files) else None
        dff_stack = calculate_dff(hi_stack, lo_stack, no_stack)
        dff_values.append(dff_stack)

    # Calculate the average of all dF/F stacks
    avg_dff = np.mean(dff_values, axis=0)
    
    # Calculate activation area from the averaged dF/F
    activation_area = calculate_activation_area(avg_dff)

    return activation_area

# Main function to process the root directory and generate the Excel sheet
def process_root_directory(root_folder, output_excel_path):
    # Define the fixed rows (uA values) for the Excel sheet
    uA_folders = ['50uA', '60uA', '70uA', '80uA', '100uA', '200uA', '300uA', '400uA', '500uA']
    
    # Initialize the DataFrame for storing the results
    results_df = pd.DataFrame(index=uA_folders)
    
    # Traverse the root folder
    for dirpath, dirnames, _ in os.walk(root_folder):
        # Check if the current directory contains uA folders
        relevant_uA_folders = [d for d in dirnames if d.endswith('uA') and d in uA_folders]
        if relevant_uA_folders:
            folder_name = os.path.basename(dirpath)
            print(folder_name)
            folder_results = {}
            
            for uA_folder in relevant_uA_folders:
                uA_folder_path = os.path.join(dirpath, uA_folder)
                
                # Process the uA folder and calculate activation area
                activation_area = process_uA_folder(uA_folder_path)
                
                # Store the activation area in the folder results
                folder_results[uA_folder] = activation_area
            
            # Add the results for this folder to the DataFrame
            results_df[folder_name] = pd.Series(folder_results)

    # Save the results to an Excel file
    results_df.to_excel(output_excel_path)
    print(output_excel_path)



In [16]:
root_folder = '/mnt/team/Raymond Lab/Judy/dLight Experiments/Stim-Response Curves/WT'
output_excel_path = '/mnt/team/Raymond Lab/Judy/dLight Experiments/Stim-Response Curves/WT/WT_Area_results.xlsx'

# Process the directory and generate the Excel sheet
process_root_directory(root_folder, output_excel_path)

2023.08.11 (3)
2023.06.29 (3)
2023.06.29
2023.06.28 (2)
2023.06.29 (2)
2023.08.16
2023.08.11
2023.08.11 (2)
2023.08.16 (2)
/mnt/team/Raymond Lab/Judy/dLight Experiments/Stim-Response Curves/WT/WT_Area_results.xlsx


### Plotting the results for the stim-response experiments

This section creates the excel spreadsheets AND generates plots. Plot generation (due to the type of 3d Plots and the transparency modification is computationally expensive. Only run this part if you really need the plots as well.

In [1]:
import os
import re
import numpy as np
import tifffile as tiff
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize

# Helper function for natural sorting of file names
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

# Function to calculate dF/F
def calculate_dff(hi_stack, lo_stack, no_stack):
    if lo_stack is not None:
        dff_stack = (hi_stack - no_stack) / no_stack + (lo_stack - no_stack) / no_stack
        dff_stack /= 2  # Averaging the two dff calculations
    else:
        dff_stack = (hi_stack - no_stack) / no_stack
    return dff_stack

# Function to calculate activation area based on dF/F thresholding
def calculate_activation_area(dff_stack):
    std_dev = np.std(dff_stack[:30], axis=0)
    threshold = 4 * std_dev
    activated_area = np.sum(dff_stack > threshold, axis=(1, 2))
    total_activation_area = np.sum(activated_area)
    return total_activation_area

# 3D plot function for the averaged dF/F stack
def plot_dff_stack_with_correct_transparency(sweep_stack, threshold, title, save_path):
    frames, height, width = sweep_stack.shape
    x = np.arange(width)
    y = np.arange(height)
    X, Y = np.meshgrid(x, y)

    # Create the 3D plot
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Normalize the colormap to the desired range (0 to 0.1)
    norm = Normalize(vmin=0, vmax=0.1)

    # Define z scaling to improve the spread of the plot in the Z-axis (time)
    z_scale = 1  

    frame_step = 5  # Skipping frames to speed up (adjust based on your data size)
    for t in range(0, frames, frame_step):
        frame = sweep_stack[t]

        # Apply transparency: Create a facecolors array based on dF/F values and mask
        facecolors = cm.jet(norm(frame))
        
        # Masking: Set alpha to 0 for values below the threshold (transparent)
        below_threshold_mask = frame < threshold
        above_threshold_mask = frame >= threshold
        way_above_threshold_mask = frame >= 3*threshold
        facecolors[below_threshold_mask, 3] = 0  # Set alpha channel to 0 where masked
        facecolors[above_threshold_mask, 3] = 0.6
        facecolors[way_above_threshold_mask, 3] = 0.9
        # Use plot_surface with custom facecolors
        ax.plot_surface(X, np.ones((height, width)) * t * z_scale, Y,
                        facecolors=facecolors, rstride=1, cstride=1, 
                        linewidth=0, antialiased=True)

    # Set axis labels and title
    ax.set_ylabel('Time (Frames)')
    ax.set_title(title)

    # Adjust the Z-axis limit to make sure it looks well-proportioned
    ax.set_ylim(0, frames * z_scale)

    # Add color bar with the new limits 
    mappable = cm.ScalarMappable(norm=norm, cmap='jet')
    fig.colorbar(mappable, ax=ax, shrink=0.5, aspect=5, label="dF/F Value")

    # Save the plot
    plt.savefig(save_path, dpi=300)
    print(save_path)
    # Show the plot (optional, can be removed if you only want to save)
    plt.close(fig)

# Function to process the uA folder, calculate average dF/F and activation area, and plot 3D dF/F stack
def process_uA_folder(uA_folder_path):
    # Gather and sort hi, lo, and no tiff files using natural sorting
    hi_files = sorted([f for f in os.listdir(uA_folder_path) if f.startswith('hi') and f.endswith('.tif')], key=natural_sort_key)
    lo_files = sorted([f for f in os.listdir(uA_folder_path) if f.startswith('lo') and f.endswith('.tif')], key=natural_sort_key)
    no_files = sorted([f for f in os.listdir(uA_folder_path) if f.startswith('no') and f.endswith('.tif')], key=natural_sort_key)

    dff_values = []
    for i, hi_file in enumerate(hi_files):
        hi_stack = np.array(tiff.imread(os.path.join(uA_folder_path, hi_file)),dtype=np.float32)
        no_stack = np.array(tiff.imread(os.path.join(uA_folder_path, no_files[i])),dtype=np.float32)

        lo_stack = np.array(tiff.imread(os.path.join(uA_folder_path, lo_files[i])),dtype=np.float32) if i < len(lo_files) else None
        dff_stack = calculate_dff(hi_stack, lo_stack, no_stack)
        dff_values.append(dff_stack)

    # Calculate the average of all dF/F stacks
    avg_dff = np.mean(dff_values, axis=0)
    
    # Calculate activation area from the averaged dF/F
    activation_area = calculate_activation_area(avg_dff)

    # Plot and save the 3D plot of the average dF/F
    std_dev = np.std(avg_dff[:30], axis=0)
    threshold = 4 * std_dev
    plot_save_path = os.path.join(os.path.dirname(uA_folder_path),"plots", f"{os.path.basename(uA_folder_path)}_average_dff_plot.png")
    plot_dff_stack_with_correct_transparency(avg_dff, threshold, f"{uA_folder_path} Average dF/F Plot", plot_save_path)

    return activation_area

# Main function to process the root directory, plot dF/F and generate the Excel sheet
def process_root_directory_with_plots(root_folder, output_excel_path):
    # Define the fixed rows (uA values) for the Excel sheet
    uA_folders = ['50uA', '60uA', '70uA', '80uA', '100uA', '200uA', '300uA', '400uA', '500uA']
    
    # Initialize the DataFrame for storing the results
    results_df = pd.DataFrame(index=uA_folders)
    
    # Traverse the root folder
    for dirpath, dirnames, _ in os.walk(root_folder):
        # Check if the current directory contains uA folders
        relevant_uA_folders = [d for d in dirnames if d.endswith('uA') and d in uA_folders]
        if relevant_uA_folders:
            folder_name = os.path.basename(dirpath)
            folder_results = {}
            
            for uA_folder in relevant_uA_folders:
                uA_folder_path = os.path.join(dirpath, uA_folder)
                
                # Process the uA folder, calculate activation area and generate plot
                activation_area = process_uA_folder(uA_folder_path)
                
                # Store the activation area in the folder results
                folder_results[uA_folder] = activation_area
            
            # Add the results for this folder to the DataFrame
            results_df[folder_name] = pd.Series(folder_results)

    # Save the results to an Excel file
    results_df.to_excel(output_excel_path)




In [2]:
root_folder = '/mnt/team/Raymond Lab/Judy/dLight Experiments/Stim-Response Curves/WT'
output_excel_path = '/mnt/team/Raymond Lab/Judy/dLight Experiments/Stim-Response Curves/WT/test.xlsx'

# Process the directory, generate plots and the Excel sheet
process_root_directory_with_plots(root_folder, output_excel_path)

/mnt/team/Raymond Lab/Judy/dLight Experiments/Stim-Response Curves/WT/2023.08.11 (3)/plots/70uA_average_dff_plot.png
/mnt/team/Raymond Lab/Judy/dLight Experiments/Stim-Response Curves/WT/2023.08.11 (3)/plots/60uA_average_dff_plot.png
/mnt/team/Raymond Lab/Judy/dLight Experiments/Stim-Response Curves/WT/2023.08.11 (3)/plots/80uA_average_dff_plot.png
/mnt/team/Raymond Lab/Judy/dLight Experiments/Stim-Response Curves/WT/2023.08.11 (3)/plots/400uA_average_dff_plot.png
/mnt/team/Raymond Lab/Judy/dLight Experiments/Stim-Response Curves/WT/2023.08.11 (3)/plots/300uA_average_dff_plot.png
/mnt/team/Raymond Lab/Judy/dLight Experiments/Stim-Response Curves/WT/2023.08.11 (3)/plots/100uA_average_dff_plot.png
/mnt/team/Raymond Lab/Judy/dLight Experiments/Stim-Response Curves/WT/2023.08.11 (3)/plots/50uA_average_dff_plot.png
/mnt/team/Raymond Lab/Judy/dLight Experiments/Stim-Response Curves/WT/2023.08.11 (3)/plots/200uA_average_dff_plot.png
/mnt/team/Raymond Lab/Judy/dLight Experiments/Stim-Response 